In [34]:
import os
import sys
import pdb
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import torch
from torch import nn
from torch.cuda.amp import GradScaler, autocast
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Adam, lr_scheduler
import torchvision
import torchvision.transforms as T

from sys import platform

DEVICE = 'mps' if platform == 'darwin' else 'cuda'

In [2]:
def save_model(model, i):
    sd = model.state_dict()
    path = os.path.join(
        # '/Users/georgestoica/Downloads',
        '/srv/share/gstoica3/checkpoints/REPAIR/',
        '%s.pth.tar' % i
    )
    torch.save(model.state_dict(), path)

def load_model(model, i):
    path = os.path.join(
        # '/Users/georgestoica/Downloads',
        '/srv/share/gstoica3/checkpoints/REPAIR/',
        '%s.pth.tar' % i
    )
    sd = torch.load(path, map_location=torch.device(DEVICE))
    model.load_state_dict(sd)

In [25]:
CIFAR_MEAN = [125.307, 122.961, 113.8575]
CIFAR_STD = [51.5865, 50.847, 51.255]
normalize = T.Normalize(np.array(CIFAR_MEAN)/255, np.array(CIFAR_STD)/255)
denormalize = T.Normalize(-np.array(CIFAR_MEAN)/np.array(CIFAR_STD), 255/np.array(CIFAR_STD))

train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    normalize,
])
test_transform = T.Compose([
    T.ToTensor(),
    normalize,
])
train_dset = torchvision.datasets.CIFAR10(root='/tmp', train=True,
                                        download=True, transform=train_transform)
test_dset = torchvision.datasets.CIFAR10(root='/tmp', train=False,
                                        download=True, transform=test_transform)

class_idxs = np.arange(10)
np.random.shuffle(class_idxs)
model1_classes = class_idxs[:5]
model2_classes = class_idxs[5:]

valid_examples1 = [i for i, (_, label) in tqdm(enumerate(train_dset)) if label in model1_classes]
valid_examples2 = [i for i, (_, label) in tqdm(enumerate(train_dset)) if label in model2_classes]

assert len(set(valid_examples1).intersection(set(valid_examples2))) == 0, 'sets should be disjoint'

train_aug_loader1 = torch.utils.data.DataLoader(
    torch.utils.data.Subset(train_dset, valid_examples1), batch_size=500, shuffle=True, num_workers=8
)
train_aug_loader2 = torch.utils.data.DataLoader(
    torch.utils.data.Subset(train_dset, valid_examples2), batch_size=500, shuffle=True, num_workers=8
)

test_loader = torch.utils.data.DataLoader(test_dset, batch_size=500, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


50000it [00:14, 3504.68it/s]
50000it [00:14, 3502.99it/s]


In [28]:
test_valid_examples1 = [i for i, (_, label) in tqdm(enumerate(test_dset)) if label in model1_classes]
test_valid_examples2 = [i for i, (_, label) in tqdm(enumerate(test_dset)) if label in model2_classes]

10000it [00:01, 5444.53it/s]
10000it [00:01, 5451.90it/s]


In [29]:
test_loader1 = torch.utils.data.DataLoader(
    torch.utils.data.Subset(test_dset, test_valid_examples1), batch_size=500, shuffle=False, num_workers=8
)
test_loader2 = torch.utils.data.DataLoader(
    torch.utils.data.Subset(test_dset, test_valid_examples2), batch_size=500, shuffle=False, num_workers=8
)

In [55]:
model1_classes, model2_classes

(array([3, 2, 0, 6, 4]), array([5, 7, 9, 8, 1]))

In [61]:
class_idxs = np.zeros(10, dtype=int)
class_idxs[model1_classes] = np.arange(5)
class_idxs[model2_classes] = np.arange(5)
class_idxs = torch.from_numpy(class_idxs)
class_idxs

tensor([2, 4, 1, 0, 4, 0, 3, 1, 3, 2])

In [36]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
#             self.shortcut = LambdaLayer(lambda x:
#                                         F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
                nn.BatchNorm2d(planes)
            )


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, w=1, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = w*16

        self.conv1 = nn.Conv2d(3, w*16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(w*16)
        self.layer1 = self._make_layer(block, w*16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, w*32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, w*64, num_blocks[2], stride=2)
        self.linear = nn.Linear(w*64, 512)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20(w=1):
    return ResNet(BasicBlock, [3, 3, 3], w=w)

In [86]:
def train(save_key, model, train_loader, test_loader, class_vectors, remap_class_idxs):
    optimizer = SGD(model.parameters(), lr=0.4, momentum=0.9, weight_decay=5e-4)
    # optimizer = Adam(model.parameters(), lr=0.05)
    
    # Adam seems to perform worse than SGD for training ResNets on CIFAR-10.
    # To make Adam work, we find that we need a very high learning rate: 0.05 (50x the default)
    # At this LR, Adam gives 1.0-1.5% worse accuracy than SGD.
    
    # It is not yet clear whether the increased interpolation barrier for Adam-trained networks
    # is simply due to the increased test loss of said networks relative to those trained with SGD.
    # We include the option of using Adam in this notebook to explore this question.

    EPOCHS = 100
    ne_iters = len(train_loader)
    lr_schedule = np.interp(np.arange(1+EPOCHS*ne_iters), [0, 5*ne_iters, EPOCHS*ne_iters], [0, 1, 0])
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_schedule.__getitem__)

    scaler = GradScaler()
    loss_fn = CrossEntropyLoss()
    
    losses = []
    for _ in tqdm(range(EPOCHS)):
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad(set_to_none=True)
            with autocast():
                encodings = model(inputs.to(DEVICE))
                normed_encodings = encodings / encodings.norm(dim=-1, keepdim=True)
                logits = (100.0 * normed_encodings @ class_vectors.T)
                remapped_labels = remap_class_idxs[labels].to(DEVICE)
                loss = loss_fn(logits, remapped_labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            losses.append(loss.item())
    print(evaluate(
        model, test_loader, 
        class_vectors=class_vectors, 
        remap_class_idxs=remap_class_idxs
    ))
    save_model(model, save_key)

In [133]:
# evaluates accuracy
def evaluate(model, loader, class_vectors, remap_class_idxs=None):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad(), autocast():
        for inputs, labels in loader:
            encodings = model(inputs.to(DEVICE))
            normed_encodings = encodings / encodings.norm(dim=-1, keepdim=True)
            outputs = normed_encodings @ class_vectors.T
            pred = outputs.argmax(dim=1)
            if remap_class_idxs is not None:
                correct += (remap_class_idxs[labels].to(DEVICE) == pred).sum().item()
            else:
                correct += (labels.to(DEVICE) == pred).sum().item()
            total += inputs.shape[0]
    return correct / total

# evaluates loss
def evaluate1(model, loader, class_vectors, remap_class_idxs):
    model.eval()
    losses = []
    pdb.set_trace()
    with torch.no_grad(), autocast():
        for inputs, labels in loader:
            encodings = model(inputs.to(DEVICE))
            normed_encodings = encodings / encodings.norm(dim=-1, keepdim=True)
            outputs = normed_encodings @ class_vectors.T
            loss = F.cross_entropy(outputs, remap_class_idxs[labels].to(DEVICE))
            losses.append(loss.item())
    return np.array(losses).mean()

In [88]:
import clip

In [7]:
test_dset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [8]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in test_dset.classes]).to(DEVICE)

In [9]:
model, preprocess = clip.load('ViT-B/32', DEVICE)

In [10]:
with torch.no_grad():
    text_features = model.encode_text(text_inputs)

In [11]:
text_features.shape

torch.Size([10, 512])

In [101]:
text_features /= text_features.norm(dim=-1, keepdim=True)

In [81]:
class_vecs1 = text_features[model1_classes]
class_vecs2 = text_features[model2_classes]
# class_vecs1 /= class_vecs1.norm(dim=-1, keepdim=True)
# class_vecs2 /= class_vecs2.norm(dim=-1, keepdim=True)

In [ ]:
text_features

In [89]:
model1 = resnet20(w=4).to(DEVICE)
model2 = resnet20(w=4).to(DEVICE)

In [90]:
train(
    f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}', 
    model=model1, 
    class_vectors=class_vecs1,
    train_loader=train_aug_loader1,
    test_loader=test_loader1,
    remap_class_idxs=class_idxs
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:42<00:00,  4.63s/it]


> /tmp/ipykernel_15816/889052149.py(12)evaluate()
     10             pred = outputs.argmax(dim=1)
     11             pdb.set_trace()
---> 12             correct += (remap_class_idxs[labels].to(DEVICE) == pred).sum().item()
     13     return correct
     14 

ipdb> pred.shape
torch.Size([500])
ipdb> pred[0]
tensor(0, device='cuda:0')
ipdb> remap_class_idxs[labels][0]
tensor(0)
ipdb> c
> /tmp/ipykernel_15816/889052149.py(11)evaluate()
      9             outputs = normed_encodings @ class_vectors.T
     10             pred = outputs.argmax(dim=1)
---> 11             pdb.set_trace()
     12             correct += (remap_class_idxs[labels].to(DEVICE) == pred).sum().item()
     13     return correct

ipdb> c
> /tmp/ipykernel_15816/889052149.py(12)evaluate()
     10             pred = outputs.argmax(dim=1)
     11             pdb.set_trace()
---> 12             correct += (remap_class_idxs[labels].to(DEVICE) == pred).sum().item()
     13     return correct
     14 

ipdb> q


In [95]:
print(
    evaluate(
        model1, test_loader1, class_vectors=class_vecs1, remap_class_idxs=class_idxs
    )/len(test_valid_examples1)
) 

0.9558


In [96]:
save_model(model1, f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}')

In [98]:
train(
    f'resnet20x4_CIFAR5_clses{model2_classes.tolist()}', 
    model=model2, 
    class_vectors=class_vecs2,
    train_loader=train_aug_loader2,
    test_loader=test_loader2,
    remap_class_idxs=class_idxs
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:43<00:00,  4.63s/it]


4863


# Combine models and evaluate performance

In [99]:
def strip_param_suffix(name):
    return name.replace('.weight', '').replace('.bias', '')

def combine_io_masks(io, param):
    mask = torch.zeros_like(param, device=param.device)
    try:
        if 'output' in io:
            mask[io['output'].view(-1) == 0] = 1.
        if 'input' in io and len(mask.shape) > 1:
            mask[:, io['input'].view(-1) == 0] = 1.
    except:
        pdb.set_trace()
    return mask

def mix_weights(model, alpha, key0, key1, module2io=None, whitelist_fn=lambda x: True):
    sd0 = torch.load(
        '/srv/share/gstoica3/checkpoints/REPAIR/%s.pth.tar' % key0, 
        map_location=torch.device(DEVICE)
    )
    sd1 = torch.load(
        '/srv/share/gstoica3/checkpoints/REPAIR/%s.pth.tar' % key1, 
        map_location=torch.device(DEVICE)
    )
    sd_alpha = {}
    for k in sd0.keys():
        param0 = sd0[k].to(DEVICE)
        param1 = sd1[k].to(DEVICE)
        sd_alpha[k] = (1 - alpha) * param0 + alpha * param1
        
        if module2io is not None:
            param_base = strip_param_suffix(k)
#             pdb.set_trace()
            mask = combine_io_masks(module2io[param_base], param1)
            sd_alpha[k][mask == 1] = param0[mask == 1].to(sd_alpha[k].dtype)
        
        if not whitelist_fn(k):
            sd_alpha[k] = param0
    model.load_state_dict(sd_alpha)

In [142]:
avg_model = resnet20(w=4).to(DEVICE)

mix_weights(
    avg_model, 
    .5, 
    f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}',
    f'resnet20x4_CIFAR5_clses{model2_classes.tolist()}'
)

In [143]:
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
) 

0.1


In [144]:
reset_bn_stats(avg_model)
print('Post-reset:')
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
)

Post-reset:
0.4192


# Combine models via permutation

In [140]:
# use the train loader with data augmentation as this gives better results
def reset_bn_stats(model, epochs=1, loader=train_aug_loader):
    # resetting stats to baseline first as below is necessary for stability
    for m in model.modules():
        if type(m) == nn.BatchNorm2d:
            m.momentum = None # use simple average
            m.reset_running_stats()
    # run a single train epoch with augmentations to recalc stats
    model.train()
    for _ in range(epochs):
        with torch.no_grad(), autocast():
            for images, _ in loader:
                output = model(images.to(DEVICE))

In [119]:
# given two networks net0, net1 which each output a feature map of shape NxCxWxH
# this will reshape both outputs to (N*W*H)xC
# and then compute a CxC correlation matrix between the outputs of the two networks
def run_corr_matrix(net0, net1, epochs=1, norm=True, loader=train_aug_loader):
    n = epochs*len(loader)
    mean0 = mean1 = std0 = std1 = None
    with torch.no_grad():
        net0.eval()
        net1.eval()
        for _ in range(epochs):
            for i, (images, _) in enumerate(tqdm(loader)):
                img_t = images.float().to(DEVICE)
                out0 = net0(img_t)
                out0 = out0.reshape(out0.shape[0], out0.shape[1], -1).permute(0, 2, 1)
                out0 = out0.reshape(-1, out0.shape[2])#.double()

                out1 = net1(img_t)
                out1 = out1.reshape(out1.shape[0], out1.shape[1], -1).permute(0, 2, 1)
                out1 = out1.reshape(-1, out1.shape[2])#.double()

                mean0_b = out0.mean(dim=0)
                mean1_b = out1.mean(dim=0)
                std0_b = out0.std(dim=0)
                std1_b = out1.std(dim=0)
                outer_b = (out0.T @ out1) / out0.shape[0]

                if i == 0:
                    mean0 = torch.zeros_like(mean0_b)
                    mean1 = torch.zeros_like(mean1_b)
                    std0 = torch.zeros_like(std0_b)
                    std1 = torch.zeros_like(std1_b)
                    outer = torch.zeros_like(outer_b)
                mean0 += mean0_b / n
                mean1 += mean1_b / n
                std0 += std0_b / n
                std1 += std1_b / n
                outer += outer_b / n

    cov = outer - torch.outer(mean0, mean1)
    if norm:
        corr = cov / (torch.outer(std0, std1) + 1e-4)
        return corr
    else:
        return cov

def compute_perm_map(corr_mtx):
    # sort the (i, j) channel pairs by correlation
    nchan = corr_mtx.shape[0]
    triples = [(i, j, corr_mtx[i, j].item()) for i in range(nchan) for j in range(nchan)]
    triples = sorted(triples, key=lambda p: -p[2])
    # greedily find a matching
    perm_d = {}
    for i, j, c in triples:
        if not (i in perm_d.keys() or j in perm_d.values()):
            perm_d[i] = j
    perm_map = torch.tensor([perm_d[i] for i in range(nchan)])

    # qual_map will be a permutation of the indices in the order
    # of the quality / degree of correlation between the neurons found in the permutation.
    # this just for visualization purposes.
    qual_l = [corr_mtx[i, perm_map[i]].item() for i in range(nchan)]
    qual_map = torch.tensor(sorted(range(nchan), key=lambda i: -qual_l[i]))

    return perm_map, qual_map

def get_layer_perm1(corr_mtx, method='max_weight', vizz=False, prune_threshold=-torch.inf):
    if method == 'greedy':
        perm_map, qual_map = compute_perm_map(corr_mtx)
        if vizz:
            corr_mtx_viz = (corr_mtx[qual_map].T[perm_map[qual_map]]).T
            viz(corr_mtx_viz)
    elif method == 'max_weight':
        corr_mtx_a = corr_mtx.cpu().numpy()
        row_ind, col_ind = scipy.optimize.linear_sum_assignment(corr_mtx_a, maximize=True)
        assert (row_ind == np.arange(len(corr_mtx_a))).all()
        perm_map = torch.tensor(col_ind).long()
        perm_map = torch.eye(corr_mtx.shape[0], device=corr_mtx.device)[perm_map]
    else:
        raise Exception('Unknown method: %s' % method)
    
#     pdb.set_trace()
    pruned_elements = torch.from_numpy(
        corr_mtx_a[row_ind, col_ind] >= prune_threshold
    ).to(perm_map.device).to(torch.float32)
    return perm_map, pruned_elements

# returns the channel-permutation to make layer1's activations most closely
# match layer0's.
def get_layer_perm(net0, net1, method='max_weight', vizz=False, prune_threshold=-torch.inf):
    corr_mtx = run_corr_matrix(net0, net1)
    return get_layer_perm1(corr_mtx, method, vizz, prune_threshold)

# modifies the weight matrices of a convolution and batchnorm
# layer given a permutation of the output channels
def permute_output(perm_map, conv, bn):
    pre_weights = [
        conv.weight,
        bn.weight,
        bn.bias,
        bn.running_mean,
        bn.running_var,
    ]
    for w in pre_weights:
        if len(w.shape) == 4:
            transform = torch.einsum('ab,bcde->acde', perm_map, w)
        elif len(w.shape) == 2:
            transform = perm_map @ w
        else:
            transform = w @ perm_map.t()
#         assert torch.allclose(w[perm_map.argmax(-1)], transform)
        w.data = transform
#         w.data = w[perm_map]

# modifies the weight matrix of a convolution layer for a given
# permutation of the input channels
def permute_input(perm_map, after_convs):
    if not isinstance(after_convs, list):
        after_convs = [after_convs]
    post_weights = [c.weight for c in after_convs]
    for w in post_weights:
        if len(w.shape) == 4:
            transform = torch.einsum('abcd,be->aecd', w, perm_map.t())
        elif len(w.shape) == 2:
            transform = w @ perm_map.t()
    #     assert torch.allclose(w[:, perm_map.argmax(-1)], transform)
        w.data = transform
#         w.data = w[:, perm_map, :, :]

def permute_cls_output(perm_map, linear):
    for w in [linear.weight, linear.bias]:
        w.data = perm_map @ w

In [116]:
model0 = resnet20(w=4).to(DEVICE)
model1 = resnet20(w=4).to(DEVICE)
load_model(model0, f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}')
load_model(model1, f'resnet20x4_CIFAR5_clses{model2_classes.tolist()}')

print(evaluate(model0, test_loader1, class_vecs1, remap_class_idxs=class_idxs))
print(evaluate(model1, test_loader2, class_vecs2, remap_class_idxs=class_idxs))

0.9558
0.9726


In [117]:
prune_threshold = -torch.inf
from collections import defaultdict
module2io = defaultdict(lambda: dict())

In [120]:
class Subnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        self = self.model
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        return x
# corr = run_corr_matrix(Subnet(model0), Subnet(model1))
# perm_map1 = get_layer_perm1(corr)
perm_map, collapse_totals = get_layer_perm(Subnet(model0), Subnet(model1), prune_threshold=prune_threshold)
permute_output(perm_map, model1.conv1, model1.bn1)
permute_output(perm_map, model1.layer1[0].conv2, model1.layer1[0].bn2)
permute_output(perm_map, model1.layer1[1].conv2, model1.layer1[1].bn2)
permute_output(perm_map, model1.layer1[2].conv2, model1.layer1[2].bn2)
permute_input(perm_map, [model1.layer1[0].conv1, model1.layer1[1].conv1, model1.layer1[2].conv1])
permute_input(perm_map, [model1.layer2[0].conv1, model1.layer2[0].shortcut[0]])

module2io['conv1']['output'] = collapse_totals
module2io['bn1']['output'] = collapse_totals
module2io['layer1.0.conv2']['output'] = collapse_totals
module2io['layer1.0.bn2']['output'] = collapse_totals
module2io['layer1.1.conv2']['output'] = collapse_totals
module2io['layer1.1.bn2']['output'] = collapse_totals
module2io['layer1.2.conv2']['output'] = collapse_totals
module2io['layer1.2.bn2']['output'] = collapse_totals

module2io['layer1.0.conv1']['input'] = collapse_totals
module2io['layer1.1.conv1']['input'] = collapse_totals
module2io['layer1.2.conv1']['input'] = collapse_totals
module2io['layer2.0.conv1']['input'] = collapse_totals
module2io['layer2.0.shortcut.0']['input'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 18.65it/s]


In [122]:
class Subnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        self = self.model
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

perm_map, collapse_totals = get_layer_perm(Subnet(model0), Subnet(model1), prune_threshold=prune_threshold)
permute_output(perm_map, model1.layer3[0].conv2, model1.layer3[0].bn2)
permute_output(perm_map, model1.layer3[0].shortcut[0], model1.layer3[0].shortcut[1])
permute_output(perm_map, model1.layer3[1].conv2, model1.layer3[1].bn2)
permute_output(perm_map, model1.layer3[2].conv2, model1.layer3[2].bn2)

permute_input(perm_map, [model1.layer3[1].conv1, model1.layer3[2].conv1])
model1.linear.weight.data = model1.linear.weight @ perm_map.t()# [:, perm_map]
# w @ perm_map.t()

module2io['layer3.0.conv2']['output'] = collapse_totals
module2io['layer3.0.bn2']['output'] = collapse_totals
module2io['layer3.0.shortcut.0']['output'] = collapse_totals
module2io['layer3.0.shortcut.1']['output'] = collapse_totals
module2io['layer3.1.conv2']['output'] = collapse_totals
module2io['layer3.1.bn2']['output'] = collapse_totals
module2io['layer3.2.conv2']['output'] = collapse_totals
module2io['layer3.2.bn2']['output'] = collapse_totals

module2io['layer3.1.conv1']['input'] = collapse_totals
module2io['layer3.2.conv1']['input'] = collapse_totals
module2io['linear']['input'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.02it/s]


In [123]:
class Subnet(nn.Module):
    def __init__(self, model, nb=9):
        super().__init__()
        self.model = model
        self.blocks = []
        self.blocks += list(model.layer1)
        self.blocks += list(model.layer2)
        self.blocks += list(model.layer3)
        self.blocks = nn.Sequential(*self.blocks)
        self.bn1 = model.bn1
        self.conv1 = model.conv1
        self.linear = model.linear
        self.nb = nb
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.blocks[:self.nb](x)
        block = self.blocks[self.nb]
        x = block.conv1(x)
        x = block.bn1(x)
        x = F.relu(x)
        return x

blocks1 = []
blocks1 += list(model1.layer1)
blocks1 += list(model1.layer2)
blocks1 += list(model1.layer3)
blocks1 = nn.Sequential(*blocks1)

block_idx2name = {
    0: 'layer1.0',
    1: 'layer1.1',
    2: 'layer1.2',
    3: 'layer2.0',
    4: 'layer2.1',
    5: 'layer2.2',
    6: 'layer3.0',
    7: 'layer3.1',
    8: 'layer3.2'
}

for nb, (block_idx, layer_name) in zip(range(9), block_idx2name.items()):
    perm_map, collapse_totals = get_layer_perm(Subnet(model0, nb=nb), Subnet(model1, nb=nb))
    block = blocks1[nb]
    permute_output(perm_map, block.conv1, block.bn1)
    permute_input(perm_map, [block.conv2])
    
    module2io[layer_name + '.conv1']['output'] = collapse_totals
    module2io[layer_name + '.bn1']['output'] = collapse_totals
    module2io[layer_name + '.conv2']['output'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.15it/s]


In [124]:
save_model(model1, f'resnet20x4_CIFAR5_perm1_{prune_threshold}threshold')

In [136]:
avg_model = resnet20(w=4).to(DEVICE)

mix_weights(
    avg_model, 
    .5, 
    f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}',
    f'resnet20x4_CIFAR5_perm1_{prune_threshold}threshold'
)

In [137]:
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
)

0.1


In [141]:
reset_bn_stats(avg_model)
print('Post-reset:')
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
)

Post-reset:
0.5256


# Combine Models via Bipartite Matching

In [145]:
model0 = resnet20(w=4).to(DEVICE)
model1 = resnet20(w=4).to(DEVICE)
model_to_alter = resnet20(w=4).to(DEVICE)

load_model(model0, f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}')
load_model(model1, f'resnet20x4_CIFAR5_clses{model2_classes.tolist()}')
load_model(model_to_alter, f'resnet20x4_CIFAR5_clses{model2_classes.tolist()}')

print(evaluate(model0, test_loader1, class_vecs1, remap_class_idxs=class_idxs))
print(evaluate(model1, test_loader2, class_vecs2, remap_class_idxs=class_idxs))
print(evaluate(model_to_alter, test_loader2, class_vecs2, remap_class_idxs=class_idxs))


0.9558
0.9726
0.9726


In [146]:
threshold = -torch.inf

In [147]:
def get_bipartite_perm(corr, prune_threshold=-torch.inf):
    scores, idx = corr.max(0)
    valid_elements = scores >= prune_threshold
    idx = torch.where(valid_elements, idx, corr.shape[0])
    location_lookup = torch.eye(corr.shape[0]+1, corr.shape[0], device=corr.device)
    matches = location_lookup[idx]
    totals = matches.sum(0, keepdim=True)
    matches = matches / (totals + 1)
    return matches.t(), totals

def get_layer_bipartite_transform(net0, net1, prune_threshold=-torch.inf):
    corr_mtx = run_corr_matrix(net0, net1)
    return get_bipartite_perm(corr_mtx, prune_threshold=prune_threshold)

In [148]:
from collections import defaultdict
module2io = defaultdict(lambda: dict())

In [149]:
class Subnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        self = self.model
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        return x
# corr = run_corr_matrix(Subnet(model0), Subnet(model1))
# perm_map1 = get_layer_perm1(corr)
perm_map, collapse_totals = get_layer_bipartite_transform(Subnet(model0), Subnet(model1), threshold)
permute_output(perm_map, model_to_alter.conv1, model_to_alter.bn1)
permute_output(perm_map, model_to_alter.layer1[0].conv2, model_to_alter.layer1[0].bn2)
permute_output(perm_map, model_to_alter.layer1[1].conv2, model_to_alter.layer1[1].bn2)
permute_output(perm_map, model_to_alter.layer1[2].conv2, model_to_alter.layer1[2].bn2)
permute_input(perm_map, [model_to_alter.layer1[0].conv1, model_to_alter.layer1[1].conv1, model_to_alter.layer1[2].conv1])
permute_input(perm_map, [model_to_alter.layer2[0].conv1, model_to_alter.layer2[0].shortcut[0]])

module2io['conv1']['output'] = collapse_totals
module2io['bn1']['output'] = collapse_totals
module2io['layer1.0.conv2']['output'] = collapse_totals
module2io['layer1.0.bn2']['output'] = collapse_totals
module2io['layer1.1.conv2']['output'] = collapse_totals
module2io['layer1.1.bn2']['output'] = collapse_totals
module2io['layer1.2.conv2']['output'] = collapse_totals
module2io['layer1.2.bn2']['output'] = collapse_totals

module2io['layer1.0.conv1']['input'] = collapse_totals
module2io['layer1.1.conv1']['input'] = collapse_totals
module2io['layer1.2.conv1']['input'] = collapse_totals
module2io['layer2.0.conv1']['input'] = collapse_totals
module2io['layer2.0.shortcut.0']['input'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 18.77it/s]


In [150]:
class Subnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        self = self.model
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        return x

perm_map, collapse_totals = get_layer_bipartite_transform(Subnet(model0), Subnet(model1), threshold)
permute_output(perm_map, model_to_alter.layer2[0].conv2, model_to_alter.layer2[0].bn2)
permute_output(perm_map, model_to_alter.layer2[0].shortcut[0], model_to_alter.layer2[0].shortcut[1])
permute_output(perm_map, model_to_alter.layer2[1].conv2, model_to_alter.layer2[1].bn2)
permute_output(perm_map, model_to_alter.layer2[2].conv2, model_to_alter.layer2[2].bn2)

permute_input(perm_map, [model_to_alter.layer2[1].conv1, model_to_alter.layer2[2].conv1])
permute_input(perm_map, [model_to_alter.layer3[0].conv1, model_to_alter.layer3[0].shortcut[0]])

module2io['layer2.0.conv2']['output'] = collapse_totals
module2io['layer2.0.bn2']['output'] = collapse_totals
module2io['layer2.0.shortcut.0']['output'] = collapse_totals
module2io['layer2.0.shortcut.1']['output'] = collapse_totals
module2io['layer2.1.conv2']['output'] = collapse_totals
module2io['layer2.1.bn2']['output'] = collapse_totals
module2io['layer2.2.conv2']['output'] = collapse_totals
module2io['layer2.2.bn2']['output'] = collapse_totals

module2io['layer2.1.conv1']['input'] = collapse_totals
module2io['layer2.2.conv1']['input'] = collapse_totals
module2io['layer3.0.conv1']['input'] = collapse_totals
module2io['layer3.0.shortcut.0']['input'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.39it/s]


In [151]:
class Subnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        self = self.model
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

perm_map, collapse_totals = get_layer_bipartite_transform(Subnet(model0), Subnet(model1), threshold)
permute_output(perm_map, model_to_alter.layer3[0].conv2, model_to_alter.layer3[0].bn2)
permute_output(perm_map, model_to_alter.layer3[0].shortcut[0], model_to_alter.layer3[0].shortcut[1])
permute_output(perm_map, model_to_alter.layer3[1].conv2, model_to_alter.layer3[1].bn2)
permute_output(perm_map, model_to_alter.layer3[2].conv2, model_to_alter.layer3[2].bn2)

permute_input(perm_map, [model_to_alter.layer3[1].conv1, model_to_alter.layer3[2].conv1])
model_to_alter.linear.weight.data = model_to_alter.linear.weight @ perm_map.t()# [:, perm_map]
# w @ perm_map.t()

module2io['layer3.0.conv2']['output'] = collapse_totals
module2io['layer3.0.bn2']['output'] = collapse_totals
module2io['layer3.0.shortcut.0']['output'] = collapse_totals
module2io['layer3.0.shortcut.1']['output'] = collapse_totals
module2io['layer3.1.conv2']['output'] = collapse_totals
module2io['layer3.1.bn2']['output'] = collapse_totals
module2io['layer3.2.conv2']['output'] = collapse_totals
module2io['layer3.2.bn2']['output'] = collapse_totals

module2io['layer3.1.conv1']['input'] = collapse_totals
module2io['layer3.2.conv1']['input'] = collapse_totals
module2io['linear']['input'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.58it/s]


In [152]:
class Subnet(nn.Module):
    def __init__(self, model, nb=9):
        super().__init__()
        self.model = model
        self.blocks = []
        self.blocks += list(model.layer1)
        self.blocks += list(model.layer2)
        self.blocks += list(model.layer3)
        self.blocks = nn.Sequential(*self.blocks)
        self.bn1 = model.bn1
        self.conv1 = model.conv1
        self.linear = model.linear
        self.nb = nb
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.blocks[:self.nb](x)
        block = self.blocks[self.nb]
        x = block.conv1(x)
        x = block.bn1(x)
        x = F.relu(x)
        return x

blocks1 = []
blocks1 += list(model_to_alter.layer1)
blocks1 += list(model_to_alter.layer2)
blocks1 += list(model_to_alter.layer3)
blocks1 = nn.Sequential(*blocks1)

block_idx2name = {
    0: 'layer1.0',
    1: 'layer1.1',
    2: 'layer1.2',
    3: 'layer2.0',
    4: 'layer2.1',
    5: 'layer2.2',
    6: 'layer3.0',
    7: 'layer3.1',
    8: 'layer3.2'
}

for nb, (block_idx, layer_name) in zip(range(9), block_idx2name.items()):
    perm_map, collapse_totals = get_layer_bipartite_transform(
        Subnet(model0, nb=nb), Subnet(model1, nb=nb), threshold
    )
    block = blocks1[nb]
    permute_output(perm_map, block.conv1, block.bn1)
    permute_input(perm_map, [block.conv2])
    
    module2io[layer_name + '.conv1']['output'] = collapse_totals
    module2io[layer_name + '.bn1']['output'] = collapse_totals
    module2io[layer_name + '.conv2']['output'] = collapse_totals

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.95it/s]


In [153]:
save_model(model_to_alter, f'resnet20x4_CIFAR5_bipartite_{prune_threshold}threshold')

In [154]:
avg_model = resnet20(w=4).to(DEVICE)

mix_weights(
    avg_model, 
    .5, 
    f'resnet20x4_CIFAR5_clses{model1_classes.tolist()}',
    f'resnet20x4_CIFAR5_bipartite_{prune_threshold}threshold'
)

print('Pre-reset:')
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
)
reset_bn_stats(avg_model)
print('Post-reset:')
print(
    evaluate(
        avg_model, 
        test_loader, 
        class_vectors=text_features
    )
)

Pre-reset:
0.1
Post-reset:
0.4778


0.1


Post-reset:
0.5256
